In [ ]:
!git clone https://github.com/VainF/DeepLabV3Plus-Pytorch

Cloning into 'DeepLabV3Plus-Pytorch'...
remote: Enumerating objects: 705, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 705 (delta 25), reused 23 (delta 9), pack-reused 653
Receiving objects: 100% (705/705), 8.26 MiB | 7.03 MiB/s, done.
Resolving deltas: 100% (380/380), done.


In [ ]:
!pip install visdom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 33.6 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 
  Created wheel for visdom: filename=visdom-0.2.3-py3-none-any.whl size=1417107 sha256=14e64027f8898aea5b87bf2d57fc2ee4b29d9c5c5f815aaa0f3c02ffb04049a1
  Stored in directory: /root/.cache/pip/wheels/4d/73/32/0bbe55d2dccb9d80d3f020f474c15a5a1eef232817dcebe776
Successfully built visdom


In [ ]:
!python '/content/DeepLabV3Plus-Pytorch/predict.py' --input /content/drive/MyDrive/Dataset/Dark_Zurich_val_anon/rgb_anon/val/night/GOPR0356  --dataset cityscapes --model deeplabv3plus_mobilenet --ckpt /content/best_deeplabv3plus_mobilenet_cityscapes_os16.pth --save_val_results_to /content/test_results

Device: cuda
Resume model from /content/best_deeplabv3plus_mobilenet_cityscapes_os16.pth
100% 50/50 [01:08<00:00,  1.37s/it]


In [ ]:
import cv2
import numpy as np
from keras import backend as K

def iou(pred1, pred2):
  intersection = np.logical_and(pred1, pred2)
  union = np.logical_or(pred1, pred2)
  iou_score = np.sum(intersection) / np.sum(union)
  return iou_score

In [ ]:
def dice_coef(mask1, mask2, smooth=1):
  mask1.squeeze()
  mask2.squeeze
  intersect = np.sum(mask1*mask2)
  fsum = np.sum(mask1)
  ssum = np.sum(mask2)
  dice = (2 * intersect ) / (fsum + ssum)
  dice = np.mean(dice)
  dice = round(dice, 3) # for easy reading
  return dice

In [ ]:
import os
import cv2
from statistics import mean
from google.colab.patches import cv2_imshow

night_pred_dir = '/content/drive/MyDrive/Dataset/Dark_Zurich_val_anon/night_pred'
day_pred_dir = '/content/drive/MyDrive/Dataset/Dark_Zurich_val_anon/day_pred'
true_imgs_dir = '/content/drive/MyDrive/Dataset/Dark_Zurich_val_anon/gt/val/night/GOPR0356/labeled'
dice_coeffs_DvT = []
dice_coeffs_NvT = []
IoUs_DvT = []
IoUs_NvT = []

for images in os.listdir(day_pred_dir):
    night_images = str(images)
    night_images = night_images.replace("_ref", "")
    # check if the image ends with png
    true_imgs = [filename for filename in os.listdir(true_imgs_dir) if filename.startswith(images[:-16])][0]
    
    day_pred = cv2.imread(os.path.join(day_pred_dir, images))
    night_pred = cv2.imread(os.path.join(night_pred_dir, night_images))
    true_imgs = cv2.imread(os.path.join(true_imgs_dir, true_imgs))

    #dice_coeffs_DvT.append(dice_coef(day_pred, true_imgs))
    #IoUs_DvT.append(iou(true_imgs, day_pred))

    dice_coeffs_NvT.append(dice_coef(day_pred, night_pred))
    IoUs_NvT.append(iou(day_pred, night_pred))
print('Dice Coeff NvT : %s ' %mean(dice_coeffs_NvT))
#print('Dice Coeff DvT : %s ' %mean(dice_coeffs_DvT))
print('IoU NvT : %s ' %mean(IoUs_NvT))
#print('IoU NvT : %s ' %mean(IoUs_DvT))

Dice Coeff NvT : 0.59718 
IoU NvT : 0.9793993347827878 


In [ ]:
img = cv2.imread('/content/night_pred/GOPR0356_frame_000321_rgb_anon.png')#/content/day_pred/GOPR0356_frame_000321_ref_rgb_anon.png')
print(np.unique(img.reshape(-1, img.shape[2]), axis=0))

[[ 30 170 250]
 [ 32  11 119]
 [ 35 142 107]
 [ 70   0   0]
 [ 70  70  70]
 [128  64 128]
 [142   0   0]
 [152 251 152]
 [153 153 153]
 [153 153 190]
 [156 102 102]
 [232  35 244]]
